In [0]:
from azure.storage.filedatalake import DataLakeServiceClient

# Replace with your details
storage_account_name = "retaildatasetsblob"
storage_account_key = "cRoMjWAkykST/Jn+RfOFmMKqfeQ6LEVlVbkUqRkNH/bUMAI0iUnPm7Lw8YHHhXM6Wjzr0ogqaFmo+AStAhgIcg=="

# Connect to ADLS
service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net",
    credential=storage_account_key,
    api_version="2023-11-03"  # Use the correct supported API version
)

# List Containers
containers = service_client.list_file_systems()
for container in containers:
    print(container.name)


amzecomdata
globalmartmarketingdata
inventorydata
optimalchannel


In [0]:
import os
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

def read_csv_from_blob(storage_account_name, container_name, file_name, storage_account_key=None):
    """
    Read a CSV file from Azure Blob Storage using Python and return a Pandas DataFrame.

    :param storage_account_name: Azure storage account name.
    :param container_name: Blob container name.
    :param file_name: Name of the file in the container.
    :param storage_account_key: Storage account access key.
    :return: Pandas DataFrame.
    """
    
    if not storage_account_key:
        # Try to get the key from environment variables if not provided
        storage_account_key = os.environ.get('AZURE_STORAGE_KEY')
        
    if not storage_account_key:
        raise ValueError("Storage account key must be provided either as a parameter or as an environment variable 'AZURE_STORAGE_KEY'")
    
    try:
        # Create a connection string
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
        
        # Create the BlobServiceClient
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        
        # Get the container client
        container_client = blob_service_client.get_container_client(container_name)
        
        # Get the blob client
        blob_client = container_client.get_blob_client(file_name)
        
        # Download the blob content
        download_stream = blob_client.download_blob()
        
        # Convert the content to a DataFrame
        content = download_stream.readall()
        df = pd.read_csv(io.BytesIO(content))
        
        return df
    
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

# Example usage:
# df = read_csv_from_blob("your_storage_account", "optimalchannel", "city.csv", storage_account_key="your_key")
# print(df.head())

In [0]:
training_data = pd.read_csv(storage_account_name=storage_account_name,
                                      container_name="globalmartmarketingdata", 
                                      file_name="PreProcessing_final_data.csv",
                                      storage_account_key=storage_account_key)
training_data.head()

,Unnamed: 0,week,sales_amount,base_price,final_price,promotion_type,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr
0,0,2023-01-01,13516527.77,101.830513,94.148539,Percentage Discount,1152.82,810.68,1122.12,707.48,11230.47,6214.43,6723.33,11311.42,703.13,3.047174,4.333516,2.471559,2.008197,2.116972
1,1,2023-01-08,95081753.02,101.830513,94.148539,Percentage Discount,7472.53,6973.85,7179.02,6834.33,11380.75,11069.60,9505.50,14004.01,6562.00,2.636847,2.732868,2.930657,3.354279,3.570124
2,2,2023-01-15,94804406.04,101.830513,94.148539,Percentage Discount,7204.29,7383.50,7185.08,6963.40,10270.30,8861.17,7836.04,14442.24,7318.19,2.679349,3.136116,2.879586,2.938546,3.776793
3,3,2023-01-22,94833974.28,101.830513,94.148539,Percentage Discount,7726.84,6522.20,7710.31,7479.41,8335.56,11601.91,6663.51,11917.89,7745.75,3.015955,3.836348,2.983655,2.858832,2.823088
4,4,2023-01-29,94806994.45,101.830513,94.148539,Percentage Discount,6987.22,6969.68,7094.25,7294.12,9575.21,7488.18,12158.00,6753.84,7044.42,2.744554,3.511152,2.338256,2.403631,3.019390


In [0]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Select features for scaling and encoding
numerical_features = ['sales_amount', 'base_price', 'final_price',
                      'facebook_spend', 'google ads_spend', 'influencer marketing_spend',
                      'instagram_spend', 'ooh_spend', 'print_spend', 'radio_spend',
                      'tv_spend', 'youtube_spend', 'facebook_ctr', 'google ads_ctr',
                      'influencer marketing_ctr', 'instagram_ctr', 'youtube_ctr']

categorical_features = ['promotion_type']

In [0]:
# Step 1: Scale numerical features
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(training_data[numerical_features])

# Convert scaled numerical data to a DataFrame
scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=numerical_features)

# Step 2: Encode categorical features
# Step 2: Encode categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_categorical_data = encoder.fit_transform(training_data[categorical_features])


# Get column names for encoded categorical features
encoded_categorical_columns = encoder.get_feature_names_out(categorical_features)

# Convert encoded categorical data to a DataFrame
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoded_categorical_columns)

# Step 3: Combine scaled numerical data and encoded categorical data
processed_df = pd.concat([scaled_numerical_df, encoded_categorical_df], axis=1)

# Display the processed DataFrame
processed_df.head()

,sales_amount,base_price,final_price,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr,promotion_type_Buy One Get One Free,promotion_type_Percentage Discount
0,-7.208240,-1.888843,-1.079989,-6.138594,-6.036483,-6.258867,-6.414760,0.452694,-1.329475,-0.967204,0.269455,-6.263100,0.408029,2.567653,-1.354243,-2.366897,-2.237695,0.0,1.0
1,0.156877,-1.888843,-1.079989,0.671809,0.104967,0.342237,-0.124517,0.511463,0.407651,0.102746,1.192669,-0.319184,-0.548539,-0.479803,-0.315938,0.701320,1.459348,0.0,1.0
2,0.131833,-1.888843,-1.079989,0.382741,0.513173,0.348841,0.007995,0.077207,-0.382501,-0.539284,1.342926,0.447983,-0.449456,0.287937,-0.431442,-0.246289,1.985146,0.0,1.0
3,0.134503,-1.888843,-1.079989,0.945865,-0.345091,0.921263,0.537766,-0.679400,0.598106,-0.990209,0.477397,0.881749,0.335249,1.621101,-0.196076,-0.427986,-0.441227,0.0,1.0
4,0.132067,-1.888843,-1.079989,0.148817,0.100812,0.249850,0.347535,-0.194618,-0.873741,1.122828,-1.293211,0.170239,-0.297450,1.001963,-1.655725,-1.465558,0.058197,0.0,1.0


In [0]:
target_column = 'sales_amount'
X = processed_df.drop(columns=[target_column])
y = processed_df[target_column]

In [0]:
from sklearn.linear_model import LinearRegression

# Separate the features (X) and target variable (y)
# Assuming 'sales_amount' is the target variable, and rest are features
target_column = 'sales_amount'
X = processed_df.drop(columns=[target_column])
y = processed_df[target_column]

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# Extract the coefficients of the model
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

coefficients



Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

,Feature,Coefficient
5,instagram_spend,0.340402
10,youtube_spend,0.245266
2,facebook_spend,0.194287
4,influencer marketing_spend,0.157081
3,google ads_spend,0.140632
11,facebook_ctr,0.110103
6,ooh_spend,0.051393
14,instagram_ctr,0.034966
8,radio_spend,0.012180
17,promotion_type_Percentage Discount,0.011617


In [0]:
import statsmodels.api as sm

X = sm.add_constant(X)

# Build the additive linear regression model
model = sm.OLS(y, X).fit()

# Print the summary of the model
model.summary()

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           sales_amount   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     41.01
Date:                Thu, 24 Apr 2025   Prob (F-statistic):           4.05e-18
Time:                        11:49:57   Log-Likelihood:                 5.3736
No. Observations:                  53   AIC:                             25.25
Df Residuals:                      35   BIC:                             60.72
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                  -0.0028      0.028     -0.100      0.921      -0.059       0.054
base_price                             -0.0177      0.055     -0.320      0.751      -0.130       0.095
final_price                             0.0083      0.064      0.129      0.898      -0.122       0.138
facebook_spend                          0.1943      0.068      2.857      0.007       0.056       0.332
google ads_spend                        0.1406      0.084      1.672      0.104      -0.030       0.311
influencer marketing_spend              0.1571      0.084      1.874      0.069      -0.013       0.327
instagram_spend                         0.3404      0.087      3.918      0.000       0.164       0.517
ooh_spend                               0.0514      0.040      1.282      0.208      -0.030       0.133
print_spend                             0.0012      0.042      0.029      0.977      -0.083       0.086
radio_spend                             0.0122      0.044      0.274      0.786      -0.078       0.102
tv_spend                                0.0073      0.042      0.174      0.863      -0.078       0.093
youtube_spend                           0.2453      0.074      3.302      0.002       0.094       0.396
facebook_ctr                            0.1101      0.042      2.609      0.013       0.024       0.196
google ads_ctr                         -0.0878      0.048     -1.810      0.079      -0.186       0.011
influencer marketing_ctr               -0.0665      0.047     -1.424      0.163      -0.161       0.028
instagram_ctr                           0.0350      0.046      0.763      0.450      -0.058       0.128
youtube_ctr                            -0.0069      0.043     -0.162      0.872      -0.094       0.080
promotion_type_Buy One Get One Free    -0.0130      0.062     -0.210      0.835      -0.139       0.113
promotion_type_Percentage Discount      0.0102      0.049      0.208      0.837      -0.090       0.110
==============================================================================
Omnibus:                        1.524   Durbin-Watson:                   1.818
Prob(Omnibus):                  0.467   Jarque-Bera (JB):                0.758
Skew:                          -0.103   Prob(JB):                        0.684
Kurtosis:                       3.549   Cond. No.                     3.28e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.32e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""